In [2]:
import numpy as np
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

# Define the dataset
dataset = {
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Hey", "Greetings", "Good day", "Howdy", "Hey there", "Hi there", "What's up", "Yo"],
      "responses": ["Hello! How can I assist you?", "Hi there! How can I help you today?", "Hey, what can I do for you?", "Greetings! How may I assist you?", "Good day! How may I help you?", "Howdy! What can I do for you?", "Hey there! How can I assist you?", "Hi there! How can I assist you?", "Hey! How may I help you?", "Yo! What can I do for you?"]
    },
    {
      "tag": "schedule_appointment",
      "patterns": ["Can I schedule a healthcare appointment?", "I'd like to book a doctor's appointment", "Schedule a medical appointment", "When can I schedule with a healthcare provider?", "Book healthcare appointment", "Set up a doctor's appointment", "I need to make a medical appointment", "I want to set a healthcare appointment", "How can I arrange a doctor's appointment?", "Can you help me with a medical appointment?"],
      "responses": ["Sure, I can help you with that.", "Of course! Let's get started. What type of appointment do you need?", "Absolutely! Let's book your appointment. What date and time would you prefer?", "Yes, I can assist you with scheduling a healthcare appointment. What date and time works for you?", "Yes, I can help you schedule a doctor's appointment. When would you like to come in?", "Certainly! Let's schedule your appointment. What time works best for you?", "Sure, I'm here to assist you with scheduling a healthcare appointment. When are you available?", "Yes, I can help you book a medical appointment. When would you like to come in?", "Of course! Let's set up your appointment. When would you like to schedule it?", "Yes, I'm here to assist you with scheduling a medical appointment. What date and time are you looking for?"]
    },
    {
      "tag": "cancel_appointment",
      "patterns": ["I need to cancel my healthcare appointment", "Cancel my doctor's appointment", "I want to reschedule my medical appointment", "Change my healthcare appointment", "I can't make it to my medical appointment", "I want to cancel my reservation with the doctor", "Can I change my appointment date with the healthcare provider?", "My plans have changed, can I reschedule my doctor's appointment?", "I need to change the time of my healthcare appointment", "I won't be able to attend my medical appointment"],
      "responses": ["Okay, let me assist you with that. Please provide me with your appointment details.", "Sure, I can help with that. Can you please provide your appointment information?", "No problem, I can help you cancel or reschedule your appointment. Could you please provide me with your appointment details?", "Absolutely, I'll assist you with canceling or rescheduling your appointment. Can you provide me with your appointment details?", "I understand. To cancel or reschedule your appointment, please provide me with your appointment details.", "Sure, I can assist you with canceling or rescheduling your appointment. Can you please provide your appointment information?", "Okay, I'll help you with that. Please provide your appointment details so I can cancel or reschedule it.", "No worries, I'll assist you with canceling or rescheduling your appointment. Can you please provide your appointment details?", "Sure, I'll help you with that. Please provide the details of your appointment so I can assist you.", "I understand, let's work on canceling or rescheduling your appointment. Could you please provide me with your appointment details?"]
    },
    {
      "tag": "confirm_appointment",
      "patterns": ["Yes", "Okay", "Sure", "That works for me", "I confirm", "Alright", "Absolutely", "Of course", "Yes, I'm available", "Yes, that's perfect"],
      "responses": ["OK", "Perfect!", "Wonderful! all set.", "Excellent!.", "Fantastic! ", "Awesome! ", "Superb!", "Great choice!", "Fantastic choice! .", "Terrific!"]
    },
    {
      "tag": "reject_appointment",
      "patterns": ["No", "Not now", "Cancel", "I changed my mind", "No, I can't make it", "I'll pass", "Not at this time", "I don't want to proceed", "I'll decide later", "Maybe another time"],
      "responses": ["No problem. Let me know if you need any further assistance.", "Okay, just let us know if you change your mind.", "No worries. Feel free to reach out if you need assistance later.", "That's alright. Let us know if you need help in the future.", "Sure, if you change your mind, feel free to contact us.", "No problem, feel free to contact us if you change your mind.", "Okay, if you change your mind, feel free to reach out to us.", "That's fine. Let us know if you need assistance in the future.", "Alright, feel free to reach out if you need assistance later.", "No problem, we're here to help if you change your mind."]
    },
    {
      "tag": "ask_for_availability",
      "patterns": ["What times are available?", "When can I schedule a healthcare appointment?", "Do you have any openings?", "Are you free today?", "Can I get a healthcare appointment today?", "What days are you available?", "When can I come in for a medical appointment?", "Can I make an appointment for today?", "What's your availability like for healthcare appointments?", "Do you have any appointments available?"],
      "responses": ["We have several time slots available. When would you like to schedule your appointment?", "Yes, we have openings today. What time works best for you?", "We have availability throughout the week. When would you like to come in?", "Yes, we have appointments available today. What time works for you?", "We have availability today. When would you like to schedule your appointment?", "We're available throughout the week. When would you like to schedule your appointment?", "We have appointments available today. What time works for you?", "Our availability varies. When would you like to come in?", "We have various time slots available. When would you like to schedule your appointment?", "Yes, we have appointments available. When would you like to come in?"]
    },
    {
      "tag": "request_reschedule",
      "patterns": ["I need to reschedule my healthcare appointment", "Can I change my healthcare appointment time?", "I want to change my medical appointment", "I need to reschedule", "Can we reschedule?", "My plans have changed, can I reschedule?", "I can't make it at this time, can we reschedule?", "Can we move my healthcare appointment to a different day?", "I want to reschedule my appointment to another time", "Can we change the time of my healthcare appointment?"],
      "responses": ["Sure, I can help you reschedule. Please provide me with your current appointment details.", "Of course! Let's find a new appointment time that works for you.", "Certainly! Let's work on rescheduling your appointment. Can you provide your current appointment details?", "No problem, let's work on rescheduling your appointment. Can you provide your current appointment details?", "Absolutely, I can help with that. Please provide your current appointment details so we can reschedule.", "Okay, let's reschedule your appointment. Please provide your current appointment details.", "Sure, I'll assist you with rescheduling. Please provide your current appointment details.", "Yes, we can reschedule. Please provide the details of your current appointment.", "I understand, let's work on rescheduling your appointment. Please provide your current appointment details.", "Yes, we can change the time of your appointment. Please provide the details of your current appointment."]
    },
    {
      "tag": "thanks",
      "patterns": ["Thank you", "Thanks a lot", "Appreciate it", "Thank you so much", "Thanks for your help", "Thank you very much", "Thanks a bunch", "Thanks for assisting me", "Thanks a ton", "I really appreciate it"],
      "responses": ["You're welcome! If you need any further assistance, feel free to ask.", "No problem! Happy to assist.", "Glad I could help! Let me know if there's anything else I can do for you.", "You're welcome! Don't hesitate to reach out if you have more questions.", "No problem at all! Feel free to ask if you need anything else.", "You're welcome! If you have any more questions, feel free to ask.", "No problem, happy to assist you! Let me know if there's anything else I can do.", "You're welcome! If you have any other questions, feel free to ask.", "You're welcome! Let me know if there's anything else I can assist you with.", "No problem, glad I could assist you!"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Goodbye", "See you later", "Bye for now", "Take care", "Until next time", "Farewell", "I have to go now", "I'm leaving now", "Gotta go", "Time for me to go"],
      "responses": ["Goodbye! Have a great day.", "Take care! Let me know if you need anything else.", "Until next time! Have a great day.", "Goodbye! If you have any questions, feel free to ask.", "Farewell! Let me know if there's anything else I can assist you with.", "Bye for now! If you need help in the future, don't hesitate to reach out.", "Goodbye! It was a pleasure assisting you.", "See you later! If you have any more questions, feel free to ask.", "Take care! Feel free to contact us if you need further assistance.", "Goodbye! Don't hesitate to reach out if you need assistance later."]
    },
    {
      "tag": "check_availability",
      "patterns": ["Are you available now?", "Is there someone I can speak to?", "Can I talk to someone?", "Can I get assistance?", "Are you open?", "Can I get help with something?", "Is there a representative available?", "Can I speak to a human?", "Can you help me?", "I need help"],
      "responses": ["Yes, I'm here to assist you. How can I help you today?", "Absolutely! What do you need assistance with?", "Yes, I'm available to assist you. What do you need help with?", "Of course! I'm here to help. What can I assist you with?", "Yes, we're open and ready to assist you. What do you need help with?", "Yes, there's a representative available to assist you. What can I help you with?", "Yes, you can speak to me. How can I assist you?", "Sure, you're speaking to a human. What do you need assistance with?", "Yes, I'm here to help. What can I assist you with?", "Certainly! How can I assist you today?"]
    },
      {
      "tag": "check_doctor_availability",
      "patterns": ["Is Dr. [doctor_name] available?", "Can I book an appointment with Dr. [doctor_name]?", "What are the available time slots for Dr. [doctor_name]?", "Is Dr. [doctor_name] accepting new patients?", "Can I see Dr. [doctor_name] today?"],
      "responses": ["Let me check Dr. [doctor_name]'s availability for you.", "I'll find out if Dr. [doctor_name] is available. Please wait a moment.", "I'll check Dr. [doctor_name]'s schedule for you.", "Dr. [doctor_name] may have availability. Let me confirm.", "I'll see if Dr. [doctor_name] has any open slots. Please hold on."]
    },
    {
      "tag": "reschedule_confirmation",
      "patterns": ["I'd like to reschedule my appointment", "Can I change my appointment time?", "I want to change my appointment to a different day", "I need to reschedule my visit", "Can we reschedule my appointment?"],
      "responses": ["Sure, I can assist you with that. Please provide your current appointment details.", "Of course, let's find a new appointment time that works for you.", "No problem, let's see if we can find another suitable appointment slot.", "Yes, I can help you reschedule. Please provide the details of your current appointment.", "Absolutely, let's work on rescheduling your appointment. Please provide your current appointment details."]
    },
    {
      "tag": "reschedule_confirmation_response",
      "patterns": ["Yes, I'd like to reschedule", "Okay, let's change the appointment time", "Yes, please find another day for my appointment", "Sure, let's reschedule", "Yes, I need to change my appointment"],
      "responses": ["Great, let's proceed with rescheduling your appointment.", "Understood, let's find a new time that works for you.", "Got it, let me find an alternative appointment date for you.", "Sure, let's work on rescheduling your appointment.", "No problem, let's change the appointment time."]
    },
    {
      "tag": "reschedule_rejection",
      "patterns": ["No, I don't want to reschedule", "I'll keep my current appointment", "No, let's keep the appointment as it is", "I'm not ready to reschedule", "No changes needed for now"],
      "responses": ["Understood, let me know if you change your mind.", "Okay, your appointment remains unchanged.", "Got it, we'll keep the appointment as scheduled.", "No problem, feel free to reach out if you decide to reschedule.", "Sure, just let us know if you need assistance later."]
    },
    {
      "tag": "request_available_timeslots",
      "patterns": ["What are the available timeslots?", "Can you provide a list of available appointments?", "When is the next available appointment?", "What times are open for booking?", "Do you have any available slots this week?"],
      "responses": ["Let me check the available timeslots for you.", "I'll provide you with a list of available appointments.", "I'll find out the next available appointment for you.", "I'll check the open timeslots for booking.", "Let me see if there are any available slots this week."]
    },
    {
      "tag": "check_doctor_availability",
      "patterns": ["Is Dr. [doctor_name] available?", "Can I book an appointment with Dr. [doctor_name]?", "What are the available time slots for Dr. [doctor_name]?", "Is Dr. [doctor_name] accepting new patients?", "Can I see Dr. [doctor_name] today?"],
      "responses": ["Let me check Dr. [doctor_name]'s availability for you.", "I'll find out if Dr. [doctor_name] is available. Please wait a moment.", "I'll check Dr. [doctor_name]'s schedule for you.", "Dr. [doctor_name] may have availability. Let me confirm.", "I'll see if Dr. [doctor_name] has any open slots. Please hold on."]
    },
    {
      "tag": "reschedule_confirmation",
      "patterns": ["I'd like to reschedule my appointment", "Can I change my appointment time?", "I want to change my appointment to a different day", "I need to reschedule my visit", "Can we reschedule my appointment?"],
      "responses": ["Sure, I can assist you with that. Please provide your current appointment details.", "Of course, let's find a new appointment time that works for you.", "No problem, let's see if we can find another suitable appointment slot.", "Yes, I can help you reschedule. Please provide the details of your current appointment.", "Absolutely, let's work on rescheduling your appointment. Please provide your current appointment details."]
    },
    {
      "tag": "reschedule_confirmation_response",
      "patterns": ["Yes, I'd like to reschedule", "Okay, let's change the appointment time", "Yes, please find another day for my appointment", "Sure, let's reschedule", "Yes, I need to change my appointment"],
      "responses": ["Great, let's proceed with rescheduling your appointment.", "Understood, let's find a new time that works for you.", "Got it, let me find an alternative appointment date for you.", "Sure, let's work on rescheduling your appointment.", "No problem, let's change the appointment time."]
    },
    {
      "tag": "reschedule_rejection",
      "patterns": ["No, I don't want to reschedule", "I'll keep my current appointment", "No, let's keep the appointment as it is", "I'm not ready to reschedule", "No changes needed for now"],
      "responses": ["Understood, let me know if you change your mind.", "Okay, your appointment remains unchanged.", "Got it, we'll keep the appointment as scheduled.", "No problem, feel free to reach out if you decide to reschedule.", "Sure, just let us know if you need assistance later."]
    },
    {
      "tag": "request_available_timeslots",
      "patterns": ["What are the available timeslots?", "Can you provide a list of available appointments?", "When is the next available appointment?", "What times are open for booking?", "Do you have any available slots this week?"],
      "responses": ["Let me check the available timeslots for you.", "I'll provide you with a list of available appointments.", "I'll find out the next available appointment for you.", "I'll check the open timeslots for booking.", "Let me see if there are any available slots this week."]
    },
    {
      "tag": "appointment_date_time_mentioned",
      "patterns": ["I'd like to schedule an appointment for [date] at [time]", "Can I book an appointment for [date]?", "Schedule an appointment for [date] at [time]", "I need to make an appointment on [date] at [time]", "Can I see Dr. [doctor_name] on [date] at [time]?"],
      "responses": ["Sure, your appointment with Dr. [doctor_name] on [date] at [time] has been scheduled.", "Your appointment for [date] at [time] with Dr. [doctor_name] is confirmed.", "Great, we've scheduled your appointment with Dr. [doctor_name] on [date] at [time].", "Confirmed! Your appointment with Dr. [doctor_name] is set for [date] at [time].", "Your appointment with Dr. [doctor_name] on [date] at [time] has been successfully booked."]
    },
    {
      "tag": "appointment_date_mentioned",
      "patterns": ["I want to schedule an appointment for [date]", "Can I book an appointment for [date]?", "Schedule an appointment for [date]", "I need to make an appointment on [date]", "Can I see Dr. [doctor_name] on [date]?"],
      "responses": ["Sure, your appointment with Dr. [doctor_name] on [date] has been scheduled.", "Your appointment for [date] with Dr. [doctor_name] is confirmed.", "Great, we've scheduled your appointment with Dr. [doctor_name] on [date].", "Confirmed! Your appointment with Dr. [doctor_name] is set for [date].", "Your appointment with Dr. [doctor_name] on [date] has been successfully booked."]
    },
    {
      "tag": "appointment_time_mentioned",
      "patterns": ["I'd like to schedule an appointment for [time]", "Can I book an appointment for [time]?", "Schedule an appointment for [time]", "I need to make an appointment at [time]", "Can I see Dr. [doctor_name] at [time]?"],
      "responses": ["Sure, your appointment with Dr. [doctor_name] at [time] has been scheduled.", "Your appointment for [time] with Dr. [doctor_name] is confirmed.", "Great, we've scheduled your appointment with Dr. [doctor_name] at [time].", "Confirmed! Your appointment with Dr. [doctor_name] is set for [time].", "Your appointment with Dr. [doctor_name] at [time] has been successfully booked."]
    },
    {
      "tag": "request_available_dates",
      "patterns": ["What dates do you have available?", "Can you provide a list of available dates?", "When are your open dates for appointments?", "Do you have any available dates this week?"],
      "responses": ["Let me check the available dates for you.", "I'll provide you with a list of available dates.", "I'll find out the open dates for appointments.", "I'll see if there are any available dates this week."]
    },
    {
      "tag": "request_available_times",
      "patterns": ["What times are available?", "Can you provide a list of available times?", "When are your open times for appointments?", "Do you have any available timeslots today?"],
      "responses": ["Let me check the available times for you.", "I'll provide you with a list of available timeslots.", "I'll find out the open times for appointments.", "I'll see if there are any available timeslots today."]
    },
    {
      "tag": "urgent_appointment",
      "patterns": ["I need to see a doctor urgently", "It's urgent, I need medical attention", "I have an urgent medical issue", "I can't wait, I need to see a doctor now", "Emergency appointment needed", "I require immediate medical assistance", "I'm in pain and need to see a doctor immediately", "It's an emergency, I need help now", "I need urgent medical care", "I can't wait, it's urgent"],
      "responses": ["I understand the urgency. Let me see what I can do to expedite your appointment.", "I'll prioritize your request for an urgent appointment. Please hold on.", "I'll do my best to get you an appointment as soon as possible. Please hold on while I check.", "I'll escalate your request for an urgent appointment. Please wait a moment.", "I'll make sure to get you the earliest available appointment. Please wait while I check for you."]
    },
    {
      "tag": "emergency_assistance",
      "patterns": ["I'm having a medical emergency", "Emergency medical assistance needed", "I'm in a critical condition", "This is an emergency situation", "I need help immediately", "Urgent medical attention required", "I'm in a life-threatening situation", "This is a medical emergency", "I need urgent medical help", "I'm facing a medical crisis"],
      "responses": ["Please dial emergency services immediately if you're in a life-threatening situation.", "For urgent medical assistance, please call emergency services or visit the nearest emergency room.", "If it's a medical emergency, please call emergency services right away for immediate help.", "I'm sorry to hear that you're in a critical condition. Please call emergency services immediately for assistance."]
    },
    {
      "tag": "greet_with_name",
      "patterns": ["Hi, I'm [name]", "Hello, I am [name]", "Hey, my name is [name]", "Greetings, I go by [name]", "Good day, call me [name]", "Howdy, I'm [name]", "Hey there, I'm known as [name]", "Hi there, you can call me [name]", "What's up, I go by [name]", "Yo, my name is [name]"],
      "responses": ["Hello, [name]! How can I assist you?", "Hi [name]! How can I help you today?", "Hey [name], what can I do for you?", "Greetings, [name]! How may I assist you?", "Good day, [name]! How may I help you?", "Howdy, [name]! What can I do for you?", "Hey there, [name]! How can I assist you?", "Hi there, [name]! How can I assist you?", "Hey [name]! How may I help you?", "Yo [name]! What can I do for you?"]
    }
  ]
}


# Collect all patterns and responses
patterns = []
responses = []
tags = []
for intent in dataset['intents']:
    patterns.extend(intent['patterns'])
    responses.extend(intent['responses'])
    tags.extend([intent['tag']] * len(intent['patterns']))

# Tokenize the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)

# Convert text sequences to integer sequences
pattern_seqs = tokenizer.texts_to_sequences(patterns)

# Pad sequences to ensure uniform length for inputs
max_len = max(len(seq) for seq in pattern_seqs)
pattern_seqs_padded = pad_sequences(pattern_seqs, maxlen=max_len)

# Convert tags to one-hot encoded vectors
tag_onehot = np.zeros((len(tags), len(set(tags))))
for i, tag in enumerate(tags):
    tag_index = list(set(tags)).index(tag)
    tag_onehot[i, tag_index] = 1

# Define model architecture
embedding_dim = 128
lstm_units = 256

model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim),
    LSTM(units=lstm_units),
    Dense(len(set(tags)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(pattern_seqs_padded, tag_onehot, epochs=100, batch_size=16)







Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0432 - loss: 3.1261    
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1145 - loss: 2.9833
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2163 - loss: 2.7975
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2479 - loss: 2.5041
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2692 - loss: 2.2848
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4475 - loss: 1.7879
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5590 - loss: 1.4507
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5832 - loss: 1.2632
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6735 - loss: 0.9994
Epoch 10/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7670 - loss: 0.7598
Epoch 11/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8157 - loss: 0.7142
Epoch 12/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accur

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_response(input_text, tokenizer, max_len, model, dataset):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq_padded = pad_sequences(input_seq, maxlen=max_len)
    intent_prob = model.predict(input_seq_padded)[0]
    predicted_intent_index = np.argmax(intent_prob)
    predicted_intent_tag = list(set(tags))[predicted_intent_index]
    responses = [intent['responses'] for intent in dataset['intents'] if intent['tag'] == predicted_intent_tag][0]
    return random.choice(responses)

def chat(tokenizer, max_len, model, dataset):
    print("Bot: Hello! How can I assist you?")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Bot: Goodbye! Have a great day.")
            break
        response = generate_response(user_input, tokenizer, max_len, model, dataset)
        print("Bot:", response)

# Start the chat
chat(tokenizer, max_len, model, dataset)


Bot: Hello! How can I assist you?


You:  Hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Bot: Hello! How can I assist you?


You:  Hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Bot: Yo! What can I do for you?


You:  I need an appointment


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Bot: I'll make sure to get you the earliest available appointment. Please wait while I check for you.


You:  Ok 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Bot: Hello! How can I assist you?
